In [1]:
import re
import math

import pandas as pd
import numpy as np
import plotly.express as px

import utilities as utils

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Intro
The purpose of this project is to build a fight prediction model. We want to prevent which fighter in each bout will win. There are a fair amount of potential predictors, which span five files downloaded from the github repo https://github.com/Greco1899/scrape_ufc_stats.

# UFC Data Model Diagram
<!-- ![alt text](ufc_data_model.png "UFC Data Model") -->
<img src="ufc_data_model.png" alt="isolated" width="600"/>


# Import Raw Data

The repo https://github.com/Greco1899/scrape_ufc_stats periodically scrapes the website http://www.ufcstats.com/ and saves the five files in the repository itself. The files were downloaded in October 2024 and stored in this repo's `raw data` folder.

## Event information

In [ ]:
# Event details: For Date mapping
df_event_details = pd.read_csv(
    'raw data/ufc_event_details.csv',
    na_values="--"
)
df_event_details['DATE'] = pd.to_datetime(df_event_details['DATE'], format="%B %d, %Y")

## Fight information

In [18]:
# Fight details: Full list of all fights (Bouts), with associated Events
df_fight_details = pd.read_csv(
    'raw data/ufc_fight_details.csv',
    na_values="--"
)

# Fight results: For Outcome and Bout-Weightclass mapping
df_fight_results = pd.read_csv(
    'raw data/ufc_fight_results.csv',
    na_values="--"
)

Now to create the Red and blue fighter columns by assuming the first-listed fighter in the `BOUT` column is the red fighter, and the second is blue. For example in the event `Jairzinho Rozenstruik  vs. Ciryl Gane`, Rozenstruik is the Red fighter and Gane is Blue.

In [19]:
# This re pattern matches the "vs." with varying amounts of whitespace on either side
df_fight_results[['FIGHTER_Red', 'FIGHTER_Blue']] = df_fight_results['BOUT'].str.split(r"\s+vs\.\s+", expand=True)

# Strip the whitespace
df_fight_results['FIGHTER_Red'] = df_fight_results['FIGHTER_Red'].str.strip()
df_fight_results['FIGHTER_Blue'] = df_fight_results['FIGHTER_Blue'].str.strip()

In [20]:
# Fight stats: stats per round and fighter for each bout
df_fight_stats = pd.read_csv(
    'raw data/ufc_fight_stats.csv',
    na_values="--"
)

## Fighter information

In [ ]:
# Fighter details: Name, nickname and URL
df_fighter_details = pd.read_csv(
    'raw data/ufc_fighter_details.csv',
    na_values="--"
)

# Fighter stats: Basic fighter stats like height, weight, stance, DOB
df_fighter_tott = pd.read_csv(
    'raw data/ufc_fighter_tott.csv',
    na_values="--"
)
df_fighter_tott['DOB'] = pd.to_datetime(df_fighter_tott['DOB'], format="%b %d, %Y")

# Create `df_fighter`
To begin we will permanently join the fighter details and fighter tott dataframes. They both have fighter as the primary key, and as shown below there is almost no variations in spelling that would prevent a join. This will simplify table management later

In [21]:
df_fighter = pd.merge(
    df_fighter_details,
    df_fighter_tott,
    on='URL',
    how='outer',
    indicator=True
)

# Only one row was in the fighter tott and not fighter details. Bringing that in to the unified df fighter table
df_fighter[df_fighter['_merge'] != 'both']

# Drop the merge indicator col
df_fighter.drop(columns=['_merge'], inplace=True)

# Overlap comparison

## Fighters

In [4]:
from pprint import pprint
import itertools
import seaborn as sns

# Create the fighter name sets from every table containing fighter name info
fighter_sets = {
    'df_fighter': set(df_fighter['FIGHTER'].dropna().unique()),
    'df_fight_results': set(df_fight_results['FIGHTER_Red'].dropna().unique()).union(set(df_fight_results['FIGHTER_Blue'].dropna().unique())),
    'df_fight_stats': set(df_fight_stats['FIGHTER'].dropna().unique()),
}

# Compare sets to each other
utils.create_set_comparison_df(fighter_sets)

,A,B,count_A,count_B,iou,perc_a_in_b,missing_from_b
0,df_fight_stats,df_fight_results,2533.0,2550.0,0.993333,1.000000,0.0
1,df_fight_results,df_fight_stats,2550.0,2533.0,0.993333,0.993333,17.0
2,df_fight_results,df_fighter,2550.0,4254.0,0.590463,0.990588,24.0
3,df_fight_stats,df_fighter,2533.0,4254.0,0.586489,0.990525,24.0
4,df_fighter,df_fight_results,4254.0,2550.0,0.590463,0.593794,1728.0
5,df_fighter,df_fight_stats,4254.0,2533.0,0.586489,0.589798,1745.0


There are somewhere between 4,240 - 4,254 fighters with personal fighter details. Only between 2,534 - 2,550 of these have participated in fights.

Ideally we would like to match `df_fighter_details` to contain the source of truth, since this table contains the `URL` column that can serve as a primary key for fighters for all fighters. We can clean any string variations such that we obtain the best possible match.

To do this however, we will need to address the following instances where `df_fighter_details` appears not to contain certain values from other tables.

### Create name fix mapping dictionary

In [5]:
# Create draft match df
df_name_map = utils.match_missings(fighter_sets, 'df_fighter').set_index('target')
df_name_map

There are  24  missing values in df_fighter


,df_fighter,df_fight_results,df_fight_stats
target,,,
Rong Zhu,Rongzhu,Rong Zhu,Rong Zhu
Bobby Green,NaN,Bobby Green,Bobby Green
Ariane Lipski,NaN,Ariane Lipski,Ariane Lipski
Michelle Waterson,Michelle Waterson-Gomez,Michelle Waterson,Michelle Waterson
Da-Un Jung,Marcos da Matta,Da-Un Jung,Da-Un Jung
Wuliji Buren,Wulijiburen,Wuliji Buren,Wuliji Buren
Katlyn Chookagian,NaN,Katlyn Chookagian,Katlyn Chookagian
Christian Duncan,Christian Leroy Duncan,Christian Duncan,Christian Duncan
Daniel Da Silva,Diego Henrique da Silva,Daniel Da Silva,Daniel Da Silva


Inspecting this table, most matches look correct, with the following exceptions. We will have to remove them manually

In [6]:
# This name was actually in the other tables but was not picked up by fuzz
df_name_map.loc['Da-Un Jung', 'df_fighter'] = 'Da Woon Jung'

# Names found by manual inspection to have been a poor match. Replace with nans for both details and tott
nans = ['Daniel Da Silva', 'Carlos Leal', 'Brianna Van Buren', 'Tecia Torres']
for name in nans:
    df_name_map.loc[name, 'df_fighter'] = np.nan

Lastly we will drop all rows with any nans from this table, since we are interested only in instances where there is a mutual match across all tables.

In [7]:
df_name_map = df_name_map.dropna().reset_index()
df_name_map

,target,df_fighter,df_fight_results,df_fight_stats
0,Rong Zhu,Rongzhu,Rong Zhu,Rong Zhu
1,Michelle Waterson,Michelle Waterson-Gomez,Michelle Waterson,Michelle Waterson
2,Da-Un Jung,Da Woon Jung,Da-Un Jung,Da-Un Jung
3,Wuliji Buren,Wulijiburen,Wuliji Buren,Wuliji Buren
4,Christian Duncan,Christian Leroy Duncan,Christian Duncan,Christian Duncan
5,Assu Almabayev,Asu Almabayev,Assu Almabayev,Assu Almabayev
6,Ian Garry,Ian Machado Garry,Ian Garry,Ian Garry
7,Na Liang,Liang Na,Na Liang,Na Liang
8,Zach Reese,Zachary Reese,Zach Reese,Zach Reese
9,Junyong Park,JunYong Park,Junyong Park,Junyong Park


### Apply name fix mapping

#### `df_fight_results`

In [8]:
# Create mapping dict from table
map_ = dict(zip(df_name_map['df_fight_results'], df_name_map['df_fighter']))

# 'Fix' both columns containing fighter names
df_fight_results['FIGHTER_Red_fix'] = df_fight_results['FIGHTER_Red'].map(map_).fillna(df_fight_results['FIGHTER_Red'])
df_fight_results['FIGHTER_Blue_fix'] = df_fight_results['FIGHTER_Blue'].map(map_).fillna(df_fight_results['FIGHTER_Blue'])

# Create subsetted dataframe to confirm results
df_sub = df_fight_results[(
    (df_fight_results['FIGHTER_Red'] != df_fight_results['FIGHTER_Red_fix']) |
    (df_fight_results['FIGHTER_Blue'] != df_fight_results['FIGHTER_Blue_fix'])
)]

print(f'{len(df_sub.index)} corrected rows:')
df_sub[['FIGHTER_Red', 'FIGHTER_Red_fix', 'FIGHTER_Blue', 'FIGHTER_Blue_fix']].head()

49 corrected rows:


,FIGHTER_Red,FIGHTER_Red_fix,FIGHTER_Blue,FIGHTER_Blue_fix
182,Julian Marquez,Julian Marquez,Zach Reese,Zachary Reese
447,Zach Reese,Zachary Reese,Cody Brundage,Cody Brundage
557,Neil Magny,Neil Magny,Ian Garry,Ian Machado Garry
591,Ode Osbourne,Ode Osbourne,Assu Almabayev,Asu Almabayev
719,Daniel Rodriguez,Daniel Rodriguez,Ian Garry,Ian Machado Garry


Lastly, remove the '_fix' suffix to create the "fixed" fight results table

In [9]:
df_fight_results = df_fight_results.drop(
    columns=['FIGHTER_Red', 'FIGHTER_Blue']
).rename(
    columns={'FIGHTER_Red_fix': 'FIGHTER_Red', 'FIGHTER_Blue_fix': 'FIGHTER_Blue'}
)

#### `df_fight_stats`

In [10]:
# Create mapping dict from table
map_ = dict(zip(df_name_map['df_fight_stats'], df_name_map['df_fighter']))

# Create 'fixed' column
df_fight_stats['FIGHTER_fix'] = df_fight_stats['FIGHTER'].map(map_).fillna(df_fight_stats['FIGHTER'])

# Create subsetted df to inspect the results
df_sub = df_fight_stats[df_fight_stats['FIGHTER'] != df_fight_stats['FIGHTER_fix']][['FIGHTER', 'FIGHTER_fix']]

print(f'{len(df_sub.index)} Corrected rows, including:')
df_sub.head(10)

164 Corrected rows, including:


,FIGHTER,FIGHTER_fix
933,Zach Reese,Zachary Reese
2280,Zach Reese,Zachary Reese
2805,Ian Garry,Ian Machado Garry
2806,Ian Garry,Ian Machado Garry
2807,Ian Garry,Ian Machado Garry
2966,Assu Almabayev,Asu Almabayev
2967,Assu Almabayev,Asu Almabayev
3591,Ian Garry,Ian Machado Garry
4030,Christian Duncan,Christian Leroy Duncan
4162,Ian Garry,Ian Machado Garry


Remove the '_fix'to create the fixed fight stats table

In [11]:
df_fight_stats = df_fight_stats.drop(
    columns=['FIGHTER']
).rename(
    columns={'FIGHTER_fix': 'FIGHTER'}
)

## Fight URL

In [12]:
fight_url_sets = {
    'df_fight_results': set(df_fight_results['URL'].dropna().unique()),
    'df_fight_details': set(df_fight_details['URL'].dropna().unique()),
}

# Compare sets to each other
utils.create_set_comparison_df(fight_url_sets)

,A,B,count_A,count_B,iou,perc_a_in_b,missing_from_b
0,df_fight_results,df_fight_details,7848.0,7861.0,0.998346,1.000000,0.0
1,df_fight_details,df_fight_results,7861.0,7848.0,0.998346,0.998346,13.0


In [13]:
fight_url_sets['df_fight_details'] - fight_url_sets['df_fight_results']

{'http://ufcstats.com/fight-details/23865dfc8361ac62',
 'http://ufcstats.com/fight-details/293c5f4c9aca5c54',
 'http://ufcstats.com/fight-details/48231fbd1af53674',
 'http://ufcstats.com/fight-details/5531317352862978',
 'http://ufcstats.com/fight-details/96704d7ba3a452d9',
 'http://ufcstats.com/fight-details/977250a28d3b8a3f',
 'http://ufcstats.com/fight-details/b65b37c0e00074d9',
 'http://ufcstats.com/fight-details/bfc9989751bd0d04',
 'http://ufcstats.com/fight-details/c53b0f678fe2f16a',
 'http://ufcstats.com/fight-details/da6b863cf684e0ae',
 'http://ufcstats.com/fight-details/dd617f6ab4916ec4',
 'http://ufcstats.com/fight-details/de89bea7b0dcd2d3',
 'http://ufcstats.com/fight-details/f7bc845ce74f5fe7'}

13 URLs are missing from `df_fight_results` that are in `df_fight_details`. It turns out these 13 rows are from the 13 bouts in a single event (UFC Fight Night: Holloway vs. the Korean Zombie).

## EVENTS
The `EVENT` column appears in four tables, and is a critical key to enable us to join fight and event information together. To run these joins we'll need to understand the various set overlaps.

In [14]:
# First the whitespace needs to be stripped from all the columns, as trailing spaces have been frequently observed.

df_event_details['EVENT'] = df_event_details['EVENT'].str.strip()
df_fight_stats['EVENT'] = df_fight_stats['EVENT'].str.strip()
df_fight_results['EVENT'] = df_fight_results['EVENT'].str.strip()
df_fight_stats['EVENT'] = df_fight_stats['EVENT'].str.strip()

In [15]:
# Create sets of the `EVENT` columns. Notice the whitespace is being stripped. Without that the overlap is poor

fight_event_sets = {
    'df_event_details': set(df_event_details['EVENT'].dropna().unique()),
    'df_fight_details': set(df_fight_details['EVENT'].dropna().unique()),
    'df_fight_results': set(df_fight_results['EVENT'].dropna().unique()),
    'df_fight_stats': set(df_fight_stats['EVENT'].dropna().unique()),
}

# Compare sets to each other
utils.create_set_comparison_df(fight_event_sets)

,A,B,count_A,count_B,iou,perc_a_in_b,missing_from_b
0,df_fight_details,df_event_details,706.0,707.0,0.998586,1.000000,0.0
1,df_fight_results,df_event_details,705.0,707.0,0.997171,1.000000,0.0
2,df_fight_results,df_fight_details,705.0,706.0,0.998584,1.000000,0.0
3,df_fight_results,df_fight_stats,705.0,705.0,1.000000,1.000000,0.0
4,df_fight_stats,df_event_details,705.0,707.0,0.997171,1.000000,0.0
5,df_fight_stats,df_fight_details,705.0,706.0,0.998584,1.000000,0.0
6,df_fight_stats,df_fight_results,705.0,705.0,1.000000,1.000000,0.0
7,df_event_details,df_fight_details,707.0,706.0,0.998586,0.998586,1.0
8,df_fight_details,df_fight_results,706.0,705.0,0.998584,0.998584,1.0
9,df_fight_details,df_fight_stats,706.0,705.0,0.998584,0.998584,1.0


Overall great overlap. There appear to be 1-2 fights not mutually common. Let's inspect those with respect to `df_fight_stats`:

In [16]:
# Create draft match df
df_name_map = utils.match_missings(fight_event_sets, 'df_fight_stats').set_index('target')
df_name_map

There are  2  missing values in df_fight_stats


,df_event_details,df_fight_details,df_fight_results,df_fight_stats
target,,,,
UFC Fight Night: Holloway vs. The Korean Zombie,UFC Fight Night: Holloway vs. The Korean Zombie,UFC Fight Night: Holloway vs. The Korean Zombie,UFC Fight Night: Ortega vs. The Korean Zombie,UFC Fight Night: Ortega vs. The Korean Zombie
UFC 294: Makhachev vs. Volkanovski 2,UFC 294: Makhachev vs. Volkanovski 2,UFC 284: Makhachev vs. Volkanovski,UFC 284: Makhachev vs. Volkanovski,UFC 284: Makhachev vs. Volkanovski


For both of the target `EVENT` values missing from `df_fight_results`, these are clearly not spelling variations. The best match identified for these was clearly a different event. Notably, the `df_fight_stats` also is missing these entries.

In summary, `df_fight_details` has two events missing across all four dataframes. `df_fight_results`. Except for this, the `EVENT` values from each table match. Because the fight results df contains the outcome of each match, there is no doing without it since our model's dependent variable will be derived from it. So the missing rows from the fight results table will have to be ignored. Once this is decided, the fight results table adds no additional information the fight results table does not contain. So it will be disregarded from further analysis.

# Export
Now we will export the dataframes that have been created or edited that will be used in downstream cleaning and analysis. But first, to summarize what dataframes will be saved back to the repo for further processing

| table name | fate | @TODO Summarized changes made in this file |
|------------|------|------|
| df_event_details | save to 'processed data/ufc_event_details.csv' | 
| df_fight_details | drop |
| df_fight_results | save to 'processed data/ufc_fight_results.csv' |
| df_fighter details, df_fighter_tott | merge and save to 'processed data/ufc_fighter.csv'|
| df_fight_stats | save to 'processed data/ufc_fight_stats.csv' |


In [17]:
# Save

df_event_details.to_csv('processed data/ufc_event_details.csv', index=False)
df_fight_results.to_csv('processed data/ufc_fight_results.csv', index=False)
df_fighter.to_csv('processed data/ufc_fighter.csv', index=False)
df_fight_stats.to_csv('processed data/ufc_fight_stats.csv', index=False)